In [1]:
import tensorflow as tf 
from tensorflow import keras 

import numpy as np 
import matplotlib.pyplot as plt

# 파일 불러오기
import os, re, glob
import cv2

image_size = 32

groups_folder_path = './IAAFT/CWT/'

X_r = []
X_a = []
y = []

for label in ['Bi', 'Tri']:
    
    for feature in ['RMS', 'Angle']:
    
    
        for top, dir, f in os.walk(groups_folder_path+label+"/"+feature+"/"):
            for filename in f:
                img = cv2.imread(groups_folder_path+label+"/"+feature+"/"+filename)
                img = cv2.resize(img, None, fx=image_size/img.shape[0], fy=image_size/img.shape[1])
                
                if feature == 'RMS':
                    X_r.append(img/256)
                    
                elif feature == 'Angle':
                    X_a.append(img/256)   
                    
                    if label == 'Bi':
                        y.append(2)
                    else:
                        y.append(3)
                        

X = []

for r, a in zip(X_r, X_a):
    img = []
    for i in range(len(r)):
        row = np.concatenate((r[i], a[i]), axis=1)
        img.append(row)
    X.append(img)
    
X = np.array(X)
y = np.array(y)


# train, test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# validation split

X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.1, random_state=10, stratify=y_train)

AlexNet = keras.models.Sequential([
        keras.layers.Conv2D(256, (6, 6), activation='relu', input_shape=X_train[0].shape), 
        keras.layers.MaxPooling2D( (2, 2) ), 
        keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'), 
        keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'), 
        keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'), 
        keras.layers.MaxPooling2D( (2, 2) ), 
        keras.layers.Flatten(), 
        keras.layers.Dense(4096, activation='relu'), 
        keras.layers.Dense(4096, activation='relu'), 
        keras.layers.Dense(len(y_train), activation='softmax') 
])

AlexNet.summary()

AlexNet.compile(loss=keras.losses.sparse_categorical_crossentropy, 
              optimizer=keras.optimizers.SGD(learning_rate=0.01), 
              metrics=[keras.metrics.sparse_categorical_accuracy])

AlexNet_history = AlexNet.fit(X_train, y_train, epochs=30, 
                    validation_data=(X_validation, y_validation))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 43, 256)       55552     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 21, 256)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 21, 384)        885120    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 21, 384)        1327488   
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 21, 256)        884992    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 10, 256)        0         
_________________________________________________________________
flatten (Flatten)            (None, 10240)             0

In [2]:
AlexNet_history = AlexNet.fit(X_train, y_train, epochs=5, 
                    validation_data=(X_test, y_test))

Train on 14400 samples, validate on 4000 samples
Epoch 1/5
14400/14400 [==============================] - 373s 26ms/sample - loss: 0.0309 - sparse_categorical_accuracy: 0.9892 - val_loss: 0.0359 - val_sparse_categorical_accuracy: 0.9883
Epoch 2/5
14400/14400 [==============================] - 377s 26ms/sample - loss: 0.0307 - sparse_categorical_accuracy: 0.9900 - val_loss: 0.0377 - val_sparse_categorical_accuracy: 0.9887
Epoch 3/5
14400/14400 [==============================] - 374s 26ms/sample - loss: 0.0303 - sparse_categorical_accuracy: 0.9901 - val_loss: 0.0419 - val_sparse_categorical_accuracy: 0.9868
Epoch 4/5
14400/14400 [==============================] - 362s 25ms/sample - loss: 0.0305 - sparse_categorical_accuracy: 0.9901 - val_loss: 0.0442 - val_sparse_categorical_accuracy: 0.9850
Epoch 5/5
14400/14400 [==============================] - 362s 25ms/sample - loss: 0.0288 - sparse_categorical_accuracy: 0.9911 - val_loss: 0.0342 - val_sparse_categorical_accuracy: 0.9887
